# Notebook's content

1. Getting ready (a.k.a. loading data and packages)
2. Creating dataset&dataloader objects
3. Creating neural network model
4. Training model

# Getting ready

In [ ]:
%%bash
conda install -c conda-forge timm -y
conda install -c conda-forge gdcm -y
pip install --upgrade torchmetrics

In [ ]:
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from skimage import exposure, io
import time
import timm
import torch
from torch import nn
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, F1, MatthewsCorrcoef
from torchvision import transforms
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
train_study = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
train_study.head()

In [ ]:
train_study.rename(columns={'Negative for Pneumonia': '0','Typical Appearance': '1',"Indeterminate Appearance": '2',
                   "Atypical Appearance": "3"}, inplace=True)
train_study.head()

In [ ]:
labels = []
def get_label(row):
    for c in train_study.columns:
        if row[c] == 1:
            labels.append(int(c))
            
train_study.apply(get_label, axis=1)
train_study.drop(columns=['0', '1','2', '3'], inplace=True)
train_study['label'] = labels
train_study.head()

# Dataset

In [ ]:
def get_img(path):
    im = cv2.imread(path)
    img = img[:, :, ::-1]
    img = exposure.equalize_hist(img)
    return img.astype(np.float32)

In [ ]:
class CovLungDataset(Dataset):
    def __init__(self, train_study, data_root, transforms=None, output_label=True, one_hot_label=False):
        super().__init__()
        
        self.df = train_study.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        
        self.output_label = output_label
        self.one_hot_label = one_hot_label
        
        if output_label == True:
            self.labels = self.df['label'].values
            
            if one_hot_label is True:
                self.labels = np.eye(self.df['label'].max() + 1)[self.labels]
            
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        # get labels
        if self.output_label:
            target = self.labels[index]
        img = get_img(os.path.join(self.data_root, f'{self.df.loc[index]["id"]}.png'))
    
        if self.transforms:
            img = self.transforms(image=img)['image']
                
        if self.output_label == True:
            return img, target
        else:
            return img

# Dataloader with data augmentation

In [ ]:
from albumentations import (
    CoarseDropout, CenterCrop, Compose,
    Cutout, HorizontalFlip, HueSaturationValue,
    Normalize, RandomBrightnessContrast, RandomResizedCrop,
    Resize, ShiftScaleRotate, Transopose,
    VerticalFlip,
)

from albumentations.pytorch import ToTensorV2

def get_train_transforms():
    return Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)
  
        
def get_valid_transforms():
    return Compose([
            CenterCrop(CFG['img_size'], CFG['img_size'], p=1.),
            Resize(CFG['img_size'], CFG['img_size']),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

In [ ]:
def prepare_dataloader(df, trn_idx, val_idx, data_root='../input/siimcovid19-512-img-png-600-study-png/study'):
    train_ = df.loc[trn_idx,:].reset_index(drop=True)
    valid_ = df.loc[val_idx,:].reset_index(drop=True)
        
    train_ds = CovLungDataset(train_, data_root, transforms=get_train_transforms(), output_label=True, one_hot_label=False)
    # validate on training data
    valid_ds = CovLungDataset(train_, data_root, transforms=get_valid_transforms(), output_label=True)
    
    train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=CFG['train_bs'],
        num_workers=CFG['num_workers'],
        pin_memory=False,
        drop_last=False,
        shuffle=True,        
    )
    
    val_loader = torch.utils.data.DataLoader(
        valid_ds, 
        batch_size=CFG['valid_bs'],
        num_workers=CFG['num_workers'],
        shuffle=False,
        pin_memory=False,
    )
    return train_loader, val_loader

# Neural network

In [ ]:
class CovClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
def PlotLossFun(epoch, Cur_List, IFTRAIN='TRAIN'):
    x = range(len(Cur_List))
    y = Cur_List
    
    IFTRAIN = IFTRAIN
    EPOCHNUM = str(epoch)
    
    plt.plot(x,y)
    plt.xlabel(f'EpochNum:{EPOCHNUM}')
    plt.ylabel(f'{IFTRAIN}cur')
    
    plt.savefig(f"{IFTRAIN}_epoch{EPOCHNUM}_cur.jpg")
    plt.show()

In [ ]:
def train_one_epoch(epoch, model, loss_fn, optimizer, train_loader, device, scheduler=None, schd_batch_update=False):   
    model.train()
    Temp_Loss_List = []

    t = time.time()
    running_loss = None

    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()

        with autocast():
            image_preds = model(imgs)

            loss = loss_fn(image_preds, image_labels)
            
            scaler.scale(loss).backward()

            if running_loss is None:
                running_loss = loss.item()
            else:
                running_loss = running_loss * .99 + loss.item() * .01

            if ((step + 1) %  CFG['accum_iter'] == 0) or ((step + 1) == len(train_loader)):
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad() 
                
                if scheduler is not None and schd_batch_update:
                    scheduler.step()

            description = f'epoch {epoch} loss: {running_loss:.4f}'
            Temp_Loss_List.append(running_loss)
            pbar.set_description(description)
    
    
    Train_Loss_List.append(np.mean(Temp_Loss_List))
    if epoch == CFG['epochs'] - 1:
        PlotLossFun(epoch, Train_Loss_List,IFTRAIN='TRAIN')            
    if scheduler is not None and not schd_batch_update:
        scheduler.step()
    return Train_Loss_List
        
def valid_one_epoch(epoch, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False):
    acc = Accuracy(num_classes=CFG['n_classes'], average='weighted').to(device)
    f1 = F1(num_classes=CFG['n_classes']).to(device)
    mcmc = MatthewsCorrcoef(num_classes=CFG['n_classes']).to(device)
    
    model.eval()

    t = time.time()
    loss_sum = 0
    sample_num = 0
    image_preds_all = []
    image_targets_all = []
    
    pbar = tqdm(enumerate(val_loader), total=len(val_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()
        
        image_preds = model(imgs)
        image_preds_all += torch.topk(image_preds, k=1)[1]
        image_targets_all += image_labels
        
        loss = loss_fn(image_preds, image_labels)
        
        loss_sum += loss.item() * image_labels.shape[0]
        sample_num += image_labels.shape[0]  

        description = f'epoch {epoch} loss: {loss_sum / sample_num:.4f}'
        pbar.set_description(description)
    
    image_preds_all = torch.cat(image_preds_all).to(device)
    image_targets_all = torch.stack(image_targets_all).to(device)
    acc_ = acc(image_preds_all, image_targets_all)
    f1_ = f1(image_preds_all, image_targets_all)
    mcmc_ = mcmc(image_preds_all, image_targets_all)
    
    print(f'---- Performance ----\nweighted acc: {acc_:.4f}, f1: {f1_:.4f}, mcmc: {mcmc_:.4f}')
    
    Val_Acc_List.append(acc(image_preds_all, image_targets_all))
    if epoch == CFG['epochs'] - 1:
        PlotLossFun(epoch, Val_Acc_List, IFTRAIN='VAL') 
    
    if scheduler is not None:
        if schd_loss_update:
            scheduler.step(loss_sum / sample_num)
        else:
            scheduler.step()

# Training

In [ ]:
CFG = {
    'seed': 256,
    'model_arch': 'tf_efficientnet_b4_ns',
    'img_size': 512,
    'epochs': 30,
    'train_bs': 16,
    'valid_bs': 32,
    'T_0': 10,
    'lr': 1e-4,
    'min_lr': 1e-6,
    'weight_decay': 1e-6,
    'num_workers': 4,
    # support to do batch accumulation for backprop with effectively larger batch size
    'accum_iter': 2,
    'verbose_step': 1,
    'device': 'cuda:0'
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
# for training only, need nightly build pytorch
seed_everything(CFG['seed'])

trn_idx, val_idx = next(iter(StratifiedShuffleSplit(n_splits=1, random_state=CFG['seed']).split(
    np.arange(train_study.shape[0]), train_study.label.values
)))


print(len(trn_idx), len(val_idx))
train_loader, val_loader = prepare_dataloader(train_study, trn_idx, val_idx, 
                                              data_root='../input/siimcovid19-512-img-png-600-study-png/study')
device = torch.device(CFG['device'])
model = CovClassifier(CFG['model_arch'], train_study.label.nunique(), pretrained=True).to(device)
scaler = GradScaler()

optimizer = torch.optim.Adam(model.parameters(), lr=CFG['lr'], weight_decay=CFG['weight_decay'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=CFG['T_0'], T_mult=1, eta_min=CFG['min_lr'], last_epoch=-1)

loss_tr = nn.CrossEntropyLoss().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)

global Train_Loss_List, Val_Acc_List
Train_Loss_List = []
Val_Acc_List = []

for epoch in range(CFG['epochs']):

    train_one_epoch(epoch, model, loss_tr, optimizer, train_loader, device, scheduler=scheduler, schd_batch_update=True)

    with torch.no_grad():
        valid_one_epoch(epoch, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False)

    if epoch%10 == 0:
        torch.save(model.model.state_dict(), f'efficient_net_{epoch}')

torch.save(model.model.state_dict(), f'efficient_net_{epoch}')
del model, optimizer, train_loader, val_loader, scaler, scheduler
torch.cuda.empty_cache()